In [1]:
import csv
from gmplot import gmplot
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
def represents_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [20]:
# Goal:
# 262 EAST 55 STREET, BROOKLYN NY 11203 =>
# 262 EAST 55th STREET, BROOKLYN NY 11203

ENDINGS = {
    '0': 'th',
    '1': 'st',
    '2': 'nd',
    '3': 'rd',
    '4': 'th',
    '5': 'th',
    '6': 'th',
    '7': 'th',
    '6': 'th',
    '8': 'th',
    '9': 'th',
    '11': 'th',
    '12': 'th',
    '13': 'th'
}
def convert_street(addr):
    # e.g. 
    (street, rest) = addr.split(',', 1)
    str_tokens = street.split(' ')
    for i in range(len(str_tokens)):
        if i == 0: continue
        t = str_tokens[i]
        if represents_int(t):
            if len(t) >= 2 and t[-2:] in ENDINGS: # 11, 12, 13
                str_tokens[i] += ENDINGS[t[-2:]]
            else:
                str_tokens[i] += ENDINGS[t[-1]]
    new_street = ' '.join(str_tokens)
    return new_street + ',' + rest

In [21]:
convert_street('1465 WEST 13 AVENUE, NY')

'1465 WEST 13th AVENUE, NY'

In [26]:
gl = Nominatim(user_agent='hri_ecb', timeout=10)

In [6]:
gl.geocode('1465A FLATBUSH AVENUE, NY')

Location(Flatbush Avenue, Gerritsen, BK, Kings County, NYC, New York, 11234, USA, (40.5955945, -73.9067876, 0.0))

In [8]:
geocodes = {}
with open('lat_long_all.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        street_addr = row['address'].split(',')[0]
        geocodes[street_addr] = [row['latitude'], row['longitude']]
print(len(geocodes))

7496


In [22]:
count = 0
with open('ecb_processed.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # only look at buildings with at least 5 violations
        if int(row['num_violations']) < 5:
            break
        converted = convert_street(row['address'])
        if converted.split(',')[0] in geocodes:
            count += 1
count

252

In [30]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(gl.geocode, min_delay_seconds=1)

In [32]:
count = 0
start_counting = False
points = []
to_latlong = {}
with open('ecb_lat_long_1.csv', 'w') as out:
    writer = csv.DictWriter(out, fieldnames=[
        'addr','lat','long'
    ])
    with open('ecb_processed.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # only look at buildings with at least 5 violations
            if int(row['num_violations']) < 5:
                break
                
            count += 1
            if not start_counting:
                start_counting = (row['address'] == '25-50 GILLMORE STREET, NY')
                continue
            
            addr = convert_street(row['address'])
            
            street_addr = addr.split(',')[0]
            if street_addr in geocodes:
                writer.writerow({'addr': addr,
                                 'lat': geocodes[street_addr][0],
                                 'long': geocodes[street_addr][1]})
            else:
                try:
                    loc = gl.geocode(addr)
                except GeocoderTimedOut as e:
                    print('Geo Time Out: ' + addr)
                    continue
                if not loc:
                    print('Geo error: ' + addr)
                    continue

                latlong = (loc.latitude, loc.longitude)
                writer.writerow({'addr': addr,
                                 'lat': latlong[0],
                                 'long': latlong[1]})

            if count % 100 == 0:
                print(str(count) + ' processed.')

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests